Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). On JupyterLab, you may want to hit the "Validate" button as well.

Caution: do not mess with the notebook's metadata; do not change a pre-existing cell's type; do not copy pre-existing cells (add new ones with the + button instead). This will break autograding; you will get a 0; you are warned.

<table style="width: 100%; border: none;" cellspacing="0" cellpadding="0" border="0">
  <tr>
    <td><img src="https://www.ip-paris.fr/voeux2022-ecolepolytechnique/images/logo_p.png" style="float: left; width: 100%" />
</td>
    <td><a style="font-size: 3em; text-align: center; vertical-align: middle;" href="https://moodle.polytechnique.fr/course/view.php?id=19260">[CSC2S004EP - 2024] - Introduction to Machine Learning</a>
</td>
  </tr>
</table>

# Lab Session 14: Reinforcement Learning

Jesse Read and Adrien Ehrhardt

In [1]:
import numpy as np

## Introduction

Reinforcement Learning (RL) is not so exotic when we consider that it is the minimisation of expected loss, just like the rest of machine learning. However, in RL we often talk about rewards, and gain (or, return) being the sum of rewards; therefore we want to maximise this, in expectation. 

Consider an environment described by state $s_t \in \{0,1,2,3\}$ at time $t$ (grid squares indexed from left to right), and action $a_t \in \{-1,0,+1\}$ (for moving left, or staying in the same grid square, or right) and reward $r(3,+1) = 0$ ($s=3, a=1$; and $r(\cdot,\cdot) = -1$ for all other cases):

In [2]:
class Environment:
    def __init__(self):
        self.states = [0, 1, 2, 3]
        self.actions = [-1, 0, +1]
        self.terminal_state = 3
        self.reset()

    def reset(self):
        self.state = 0
        return self.state

    def step(self, action):
        # Clip action to stay within valid actions
        if action not in self.actions:
            raise ValueError(f"Invalid action {action}, must be one of {self.actions}")
        
        # Compute next state
        next_state = np.clip(self.state + action,min(self.states),max(self.states))

        # Compute reward
        if next_state == 3 and action == 1:
            reward = 0
            done = True
        else:
            reward = -1
            done = False

        self.state = next_state
        return self.state, reward, done


We need to interact with the environment, it means we need a *policy* to decide which action to take from which state. Let's start with a random one (what other choice do we have?). 

In [3]:
def policy(s):
    return np.random.choice([-1,0,+1])

We take a **rollout** (or 'trajectory', 'episode'). 

In [4]:
env = Environment()
s = env.reset()
done = False

rollout = []

while not done:
    a = policy(s)
    s, r, done = env.step(a)
    print(f"({s}, {a}, {r})")
    rollout.append((s,a,r))

(0, 0, -1)
(0, -1, -1)
(0, 0, -1)
(0, 0, -1)
(0, -1, -1)
(0, 0, -1)
(1, 1, -1)
(0, -1, -1)
(0, 0, -1)
(1, 1, -1)
(0, -1, -1)
(0, -1, -1)
(0, 0, -1)
(1, 1, -1)
(0, -1, -1)
(0, 0, -1)
(1, 1, -1)
(1, 0, -1)
(0, -1, -1)
(0, -1, -1)
(0, -1, -1)
(0, -1, -1)
(0, -1, -1)
(1, 1, -1)
(1, 0, -1)
(1, 0, -1)
(1, 0, -1)
(1, 0, -1)
(1, 0, -1)
(1, 0, -1)
(2, 1, -1)
(2, 0, -1)
(1, -1, -1)
(1, 0, -1)
(1, 0, -1)
(1, 0, -1)
(1, 0, -1)
(2, 1, -1)
(1, -1, -1)
(0, -1, -1)
(0, 0, -1)
(0, -1, -1)
(0, -1, -1)
(0, -1, -1)
(0, 0, -1)
(0, -1, -1)
(1, 1, -1)
(1, 0, -1)
(2, 1, -1)
(1, -1, -1)
(0, -1, -1)
(0, 0, -1)
(0, -1, -1)
(0, -1, -1)
(0, 0, -1)
(1, 1, -1)
(2, 1, -1)
(3, 1, 0)


What can we *learn* from this? Complete the following code to estimate the **value** of taking action $a$ from state $s$, for all $(s,a)$ pairs that occur in the rollout. So we are estimating $Q(s,a) = \mathbb{E}_{\pi}[\sum r_t \mid s, a]$. It means: $Q(s,a)$ is the expected sum of rewards obtained from state $s$, taking action $a$, under policy $\pi$ (the random one defined above).  

In [5]:
import pandas as pd
from collections import defaultdict

# Mapping for action <-> index
actions = [-1, 0, 1]
action_to_index = {a: i for i, a in enumerate(actions)}
index_to_action = {i: a for a, i in action_to_index.items()}

def estimate_q_from_rollout(rollout, num_states=4):
    """
    Estimate Q(s,a) from a single rollout using first-visit Monte Carlo.
    Returns a NumPy array of shape (num_states, num_actions).
    """
    num_actions = len(actions)
    returns = defaultdict(list)
    visited = set()

    for t in range(len(rollout)):
        s, a, _ = rollout[t]
        key = (s, a)
        if key not in visited:
            visited.add(key)
            G = sum(r for _, _, r in rollout[t:])
            returns[key].append(G)

    Q = np.full((num_states, num_actions), np.nan)
    for (s, a), G_list in returns.items():
        a_idx = action_to_index[a]
        Q[s, a_idx] = np.mean(G_list)

    return Q

def display_q_table(Q):
    """
    Display Q-table.
    """
    df = pd.DataFrame(Q, columns=[f"${a}$" for a in actions])
    df.index.name = "State"
    return df.style.background_gradient(cmap="Greens", axis=None).format(precision=1, na_rep="")

In [6]:
Q = estimate_q_from_rollout(rollout)
display_q_table(Q)

,$-1$,$0$,$1$
State,,,
0,-56.0,-57.0,
1,-25.0,-40.0,-51.0
2,,-26.0,-27.0
3,,,0.0


Is that what you expect? If we act on *this* knowledge (current `Q`), is it better than random (i.e., better than not knowing anything / have we learned something from that rollout)?

Your task is to take the first step of iteration in this RL algorithm, and rewrite the `policy` as the greedy policy (in the box below), i.e., the policy that takes the best action `a` in a given state `s` given its knowledge in `Q`.

In [7]:
def policy(s, Q):
    """
    Greedy policy: returns the action a ∈ {-1, 0, 1} for given state s
    """
    # YOUR CODE HERE
    q_values = Q[s]
    max_q = np.nanmax(q_values)
    best_actions = [a for a_idx, a in enumerate(actions) if q_values[a_idx] == max_q]
    
    if not best_actions:
        return np.random.choice(actions)
    
    return np.random.choice(best_actions)

You would find that if you generated a new rollout with *this* policy, then recalculated the greedy policy, you would **converge to the optimal policy**. 

Afterword: The goal of RL is such that our learned policy approaches (with sufficient training data) the optimal policy (which, in the real world, we do not have). This is essentially the same goal we have in any machine learning setting (except, we call it a policy $\pi$ in RL, rather than a regression function $f$, or a classifer $h$ as in other paradigms). As elsewhere in machine learning, we invest our time to find models and algorithms which achieve better results, and more efficiently, or more explainably; but there is no single best model/algorithm.